In [1]:
import requests

# List of books to download (title: URL)
books = {
    "moby_dick.txt": "https://www.gutenberg.org/files/2701/2701-0.txt",
    "pride_prejudice.txt": "https://www.gutenberg.org/files/1342/1342-0.txt",
    "frankenstein.txt": "https://www.gutenberg.org/files/84/84-0.txt",
    "alice_wonderland.txt": "https://www.gutenberg.org/files/11/11-0.txt",
    "christmas_carol.txt": "https://www.gutenberg.org/files/98/98-0.txt",
}

# Download and store in `data`
data = ""

for name, url in books.items():
    print(f"Downloading {name}...")
    response = requests.get(url)
    if response.status_code == 200:
        text = response.text.strip()
        data += text + "\n\n"
    else:
        print(f"Failed to download {name}")

print("\n✅ Download complete. `data` now contains the combined text of all books.")


✅ Download complete. `data` now contains the combined text of all books.


## Params

In [2]:
!pip install tiktoken
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

GPT_CONFIG_211M = {
    "vocab_size": 50257,  # Vocabulary size
    "context_length": 512,  # Context length
    "emb_dim": 810,  # Embedding dimension
    "n_layers": 18,  # Number of
    "n_heads": 18,  # Number of attention heads per transformer block
    "drop_rate": 0.2,  # Dropout rate
    "qkv_bias": False,  # Query-Key-Value bias
}
device = "cuda"

## Preprocessing stuff

In [4]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            self.input_ids.append(torch.tensor(token_ids[i:i + max_length]))
            self.target_ids.append(torch.tensor(token_ids[i + 1: i + max_length + 1]))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [5]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

## Complete transformer block

In [6]:
# Multiheaded attetion mechanism. Dude, this shit was fire !
class MultiHeadAttention(nn.Module):
  def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask",torch.triu(torch.ones(context_length, context_length),diagonal=1))

  def forward(self, x):
    b, num_token, d_in = x.shape
    keys = self.W_key(x)
    queries = self.W_query(x)
    values = self.W_value(x)
    keys = keys.view(b, num_token, self.num_heads, self.head_dim)
    values = values.view(b, num_token, self.num_heads, self.head_dim)
    queries = queries.view(b, num_token, self.num_heads, self.head_dim)
    keys = keys.transpose(1,2)
    values = values.transpose(1,2)
    queries = queries.transpose(1,2)
    attn_scores = queries @ keys.transpose(2,3) # we get (..., num_token, num_token)
    masked_bool = self.mask.bool()[:num_token, :num_token]
    attn_scores.masked_fill(masked_bool, -torch.inf)
    attn_scores = attn_scores / keys.shape[-1]**0.5
    attn_weights = torch.softmax(attn_scores, dim=-1)
    attn_weights = self.dropout(attn_weights)
    context_vec = (attn_weights @ values).transpose(1,2)
    context_vec = context_vec.contiguous().view(b, num_token, self.d_out)

    return context_vec

# We normalize the layer at the last dim with mean near to 0 and variance near to 1
class LayerNorm(nn.Module):
  def __init__(self, emb_dim):
    super().__init__()
    self.eps = 1e-5
    self.scale = nn.Parameter(torch.ones(emb_dim))
    self.shift = nn.Parameter(torch.zeros(emb_dim))

  def forward(self, x):
    mean = x.mean(dim=-1, keepdim=True)
    var = x.var(dim=-1, keepdim=True, unbiased=False)
    # we do +self.eps, to let the var not be 0 and division by 0 SHOULD not be done
    norm_x = (x-mean)/torch.sqrt(var + self.eps)

    # We use scale and shift for better training and they are trainable also !!!
    return self.scale * norm_x + self.shift

# GeLU function activation
class GeLU(nn.Module):
  def __init__(self):
    super().__init__()

  # Better version of ReLU()
  def forward(self, x):
    return 0.5*x*(1+torch.tanh(torch.sqrt(torch.tensor(2/torch.pi))* (x + 0.044715*x**3)))

# The classic feed froward neura network
class FeedForward(nn.Module):
  def __init__(self, cfg):
    super().__init__()

    # Feed forward network with GeLU between 2 linear
    self.layers = nn.Sequential(
        nn.Linear(cfg["emb_dim"], 4*cfg["emb_dim"]),
        GeLU(),
        nn.Linear(4*cfg["emb_dim"], cfg["emb_dim"])
    )

  def forward(self, x):
    return self.layers(x)

In [7]:
class TransformerBlock(nn.Module):
  def __init__(self, cfg):
    super().__init__()

    # Dude, no need for the comments here. You already know. It is a transformer block bro !
    self.att = MultiHeadAttention(
        d_in=cfg["emb_dim"],
        d_out=cfg["emb_dim"],
        context_length=cfg["context_length"],
        dropout=cfg["drop_rate"],
        num_heads=cfg["n_heads"],
        qkv_bias=cfg["qkv_bias"]
    )
    self.ff = FeedForward(cfg)
    self.norm1 = LayerNorm(cfg["emb_dim"])
    self.norm2 = LayerNorm(cfg["emb_dim"])
    self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

  def forward(self,x):

    # creating shortcut from x to the first dropout layer
    shortcut = x
    x = self.norm1(x)
    x = self.att(x)
    x = self.drop_shortcut(x)
    x = x + shortcut

    # creating shortcut from first dropout to the second dropout
    shortcut = x
    x = self.norm2(x)
    x = self.ff(x)
    x = self.drop_shortcut(x)
    x = x + shortcut

    return x

In [8]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        in_idx = in_idx.long()
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

## Generating new Output Tokens

In [9]:
def generate_text_simple(model, idx, max_new_tokens, context_size):

  # idx is (batch, n_tokens) array of indices in current context
  for _ in range(max_new_tokens):
    # If LLM suports only 5 tokens, and the context size is 10, then we only use last 5 toens as context.
    idx_cond = idx[:, -context_size:]

    # Gettings the predictions
    with torch.no_grad():
      # Reshape idx_cond to (batch_size, sequence_length, emb_dim)
      # idx_cond = idx_cond.unsqueeze(-1).repeat(1 , 1, model.norm1.scale.shape[0]) # Or model.att.d_in to get the embedding dimension
      logits = model(idx_cond) # (batch, num_tokens, vocab_size)

    # We take the last row. We dont do anything to the batches neither to the last dimension of the vocabularies, but take the last row
    logits = logits[:, -1, :] # (batch, vocab_size)

    # getting probablities from the logits. We can say something like 50% chances of this, 2% chances of this...
    probs = torch.softmax(logits, dim=-1) # (batch, vocab_size)

    # We see the highest value's index
    idx_next = torch.argmax(probs, dim=-1, keepdim=True) # (batch, 1)

    # Append the predicted token_id generated to the original index
    idx = torch.cat((idx, idx_next), dim=1) # (batch, num_tokens+1)

  return idx

In [10]:
def text_to_token_ids(text, tokenizer):
  encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
  encoded_tensor = torch.tensor(encoded).unsqueeze(0)
  return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
  decoded = tokenizer.decode(token_ids.squeeze(0).tolist())
  return decoded

## Loss Function

## Training and validation

In [11]:
# Train/validation ratio
train_ratio = 0.90
split_idx = int(train_ratio * len(data))
train_data = data[:split_idx]
val_data = data[split_idx:]


torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

In [12]:
# Caluclates loss for a batch
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss

# Caluculates loss for ENTIRE data_loader which calls calc_loss_batch function inside itself
def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

In [13]:
def evaluate_model(model, train_loader, val_loader, device, eval_iter):
  # basically returns the losses for training and validation
  model.eval()
  with torch.no_grad():
    return calc_loss_loader(train_loader, model, device, eval_iter), calc_loss_loader(val_loader, model, device, eval_iter)

In [14]:
def generate_and_print_sample(model, tokenizer, device, start_context):
  # we print out, what the model is generating right now at the end of each epoch. Also, we print 50 items!
  model.eval()
  context_size = model.pos_emb.weight.shape[0]
  encoded = text_to_token_ids(start_context, tokenizer).to(device)

  with torch.no_grad():
    token_ids = generate_text_simple(
        model, encoded, 50, context_size
    )

  decoded = token_ids_to_text(token_ids, tokenizer)
  print(decoded.replace("\n", " "))
  model.train()

## **TRAINING**

In [15]:
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                       eval_freq, eval_iter, start_context, tokenizer):
    # Initialize tracking lists
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen, global_step = 0, -1
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        print(f"\n🔁 Starting epoch {epoch+1}")
        model.train()
        epoch_improved = False

        for input_batch, target_batch in train_loader:
            optimizer.zero_grad()
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward()
            optimizer.step()

            tokens_seen += input_batch.numel()
            global_step += 1

            # Optional evaluation
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)

                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

                # Check for validation improvement
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    epoch_improved = True

        # Print sample after epoch
        generate_and_print_sample(model, tokenizer, device, start_context)

        # If no improvement this epoch, skip rest
        if not epoch_improved:
            print(f"⚠️ No val loss improvement in epoch {epoch+1}, skipping to next.")
            continue

    return train_losses, val_losses, track_tokens_seen

In [16]:
import time
start_time = time.time()

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.1)

num_epochs = 10
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=50, eval_iter=5,
    start_context="I had always ", tokenizer=tokenizer
)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")


🔁 Starting epoch 1
Ep 1 (Step 000000): Train loss 9.341, Val loss 9.186
Ep 1 (Step 000050): Train loss 6.760, Val loss 6.644
Ep 1 (Step 000100): Train loss 6.139, Val loss 6.314
Ep 1 (Step 000150): Train loss 5.959, Val loss 6.117
Ep 1 (Step 000200): Train loss 5.683, Val loss 5.972
Ep 1 (Step 000250): Train loss 5.771, Val loss 5.903
Ep 1 (Step 000300): Train loss 5.579, Val loss 5.770
Ep 1 (Step 000350): Train loss 5.180, Val loss 5.704
Ep 1 (Step 000400): Train loss 5.558, Val loss 5.654
Ep 1 (Step 000450): Train loss 5.205, Val loss 5.636
Ep 1 (Step 000500): Train loss 5.555, Val loss 5.551
Ep 1 (Step 000550): Train loss 4.805, Val loss 5.566
Ep 1 (Step 000600): Train loss 5.180, Val loss 5.543
Ep 1 (Step 000650): Train loss 5.201, Val loss 5.519
Ep 1 (Step 000700): Train loss 4.999, Val loss 5.463
Ep 1 (Step 000750): Train loss 5.116, Val loss 5.459
Ep 1 (Step 000800): Train loss 5.182, Val loss 5.424
I had always                                                   

🔁 Starting epo

In [17]:
from torchsummary import summary
summary(model, (256,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Embedding-1             [-1, 256, 810]      40,708,170
         Embedding-2                  [-1, 810]         414,720
           Dropout-3             [-1, 256, 810]               0
         LayerNorm-4             [-1, 256, 810]               0
            Linear-5             [-1, 256, 810]         656,100
            Linear-6             [-1, 256, 810]         656,100
            Linear-7             [-1, 256, 810]         656,100
           Dropout-8         [-1, 18, 256, 256]               0
MultiHeadAttention-9             [-1, 256, 810]               0
          Dropout-10             [-1, 256, 810]               0
        LayerNorm-11             [-1, 256, 810]               0
           Linear-12            [-1, 256, 3240]       2,627,640
             GeLU-13            [-1, 256, 3240]               0
           Linear-14             [-1, 2

In [18]:
torch.save(model.state_dict(), "GPT2_211M.pth")

In [19]:
token_ids_to_text(
    model(
        text_to_token_ids(
            "hello, i am ",
            tokenizer
        )
    ),
    tokenizer
)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

## Post training stuff

We also use torch.multinomial() instead of torch.argmax().

And then we use torch.topk() for doing top-k sampling plus the temprature scaling

In [ ]:
def generate(model, idx, max_new_tokens, context_size, temprature=0.0, top_k=None, eos_id=None):
  for _ in range(max_new_tokens):

    # compressing idx to the context_size and basically taking last 4 tokens
    idx = idx[:, -context_size:]
    with torch.no_grad():
      # we just need vocab size and batch thing, no need to keep track of num_tokens
      logits = model(idx)[:, -1, :]

    if top_k is not None:
      # We take top k elements with highests logits score
      top_logits = torch.topk(logits, top_k)

      # Getting minimum value among three
      min_val = top_logits[:, -1]

      # replacing everything in logits to -infinity that is less than min_val
      logits = torch.where(
          condition=logits<min_val,
          torch.tensor(float(-inf)).to(logits.device),
          logits
      )

    # If temprature is above 0, we do temprature scaling
    if temprature > 0.0:
      # More temprature, high creativity. Less temprature, low creativity.
      logits = logits / temprature
      # applying softmax
      probs =  torch.softmax(logits, dim=-1)

      # Using multinomial, so that we pick randomly from the top k samples
      idx_next = torch.multinomial(probs, num_sample=1)
    else:
      # If no temprature, we use argmax
      idx_next = torch.argmax(logits, dim=-1, keep_dim=True)

    # If we see end of sequence token, we stop early
    if idx_next==eos_id:
      break

    # and we concat the new token, to the older sentence and move back or out of the for loop
    idx = torch.cat((idx, idx_next), dim=1)

    return idx